-----
# Prepare a Graph with 5 different Layouts and different number of Links
-----

In [1]:
import networkx as nx
from sklearn import preprocessing 
import pandas as pd
import json
import random
import seaborn as sns
import csv 

def hex_to_rgb(hx):
    hx = hx.lstrip('#')
    hlen = len(hx)
    rgb = tuple(int(hx[i:i+hlen//3], 16) for i in range(0, hlen, hlen//3))
    return rgb

### gradually removing edges in e.g. 5 steps

In [2]:
n=100
r=0.25

G5 = nx.random_geometric_graph(n,r) 
#G5 = nx.karate_club_graph()
G5.remove_edges_from(list(G5.edges())[1:len(G5.edges()):])
print('# of links: ', len(G5.edges()))

G4 = nx.random_geometric_graph(n,r) 
#G4 = nx.karate_club_graph()
G4.remove_edges_from(list(G4.edges())[::2])
print('# of links: ', len(G4.edges()))

G3 = nx.random_geometric_graph(n,r) 
#G3 = nx.karate_club_graph()
G3.remove_edges_from(list(G3.edges())[::3])
print('# of links: ', len(G3.edges()))

G2 = nx.random_geometric_graph(n,r) 
#G2 = nx.karate_club_graph()
G2.remove_edges_from(list(G2.edges())[::6])
print('# of links: ', len(G2.edges()))

G1 = nx.random_geometric_graph(n,r) 
#G1 = nx.karate_club_graph()
print('# of links: ', len(G1.edges()))

Graphs = [G1,G2,G3,G4,G5]

# of links:  1
# of links:  366
# of links:  568
# of links:  633
# of links:  757


In [3]:
for ix,G in enumerate(Graphs):
    # --------------------------
    # POS 
    # --------------------------
    if G.name != "Zachary's Karate Club": 
        G.name = "Geometric Graph"
    graphname = ((G.name+str(ix)).replace(' ','')).replace("'",'')
    pos = nx.spring_layout(G, dim = 3)
    coords = [i.tolist() for i in list(pos.values())]
    posG = dict(zip(pos.keys(),coords))

    nx.set_node_attributes(G, posG, name="pos")

    # --------------------------
    # CLUSTER 
    # --------------------------
    if G.name == "Zachary's Karate Club":
        dict_for_cluster = nx.get_node_attributes(G, "club")
    else: 
        assigned_groups = []
        for g in G.nodes():
            n = random.randint(0,3)
            assigned_groups.append(n)
        dict_for_cluster = dict(zip(G.nodes(), assigned_groups))
    
    assigned_groups = list(dict_for_cluster.values())
    nx.set_node_attributes(G, dict_for_cluster, name="cluster")

    # --------------------------
    # NODE COLOR  
    # --------------------------
    clustercolors = sns.color_palette("Set1").as_hex()[:len(set(assigned_groups))]
    d_clustercolors = dict(zip(set(dict_for_cluster.values()),clustercolors)) 

    d_node_colors={}
    for nodeid,group in dict_for_cluster.items():
        for clu, col in d_clustercolors.items():
            if group==clu:
                d_node_colors[nodeid] = col

    nx.set_node_attributes(G, d_node_colors, name="nodecolor")

    # --------------------------
    # NODE ANNOTATION
    # --------------------------
    l_annotations_json = []
    for i, j in zip(dict_for_cluster.values(), dict_for_cluster.keys()):
        sublist = ["cluster: " + str(i) + "; node: " + str(j)]
        l_annotations_json.append(sublist)

    d_annotations = dict(zip(G.nodes(), l_annotations_json))
    nx.set_node_attributes(G, d_annotations, name="annotation")
    
    # --------------------------
    # LINK COLOR
    # --------------------------
    linkcolor = '#909090'
    nx.set_edge_attributes(G, linkcolor, name="linkcolor")

    # --------------------------
    # GRAPH NAME
    # --------------------------
    G_json = json.dumps(nx.node_link_data(G))
    with open(graphname+".json", "w") as outfile:
        outfile.write(G_json)

In [4]:
for ix,G in enumerate(Graphs):
    graphname = ((G.name+str(ix)).replace(' ','')).replace("'",'')
    
    # NODE POSITIONS 
    pos = nx.spring_layout(G, dim = 3)
    coords = [i.tolist() for i in list(pos.values())]
    posG = dict(zip(pos.keys(),coords))

    nx.set_node_attributes(G, posG, name="pos")
    df_nodepos = pd.DataFrame()
    df_nodepos['x']=[i[0] for i in posG.values()]
    df_nodepos['y']=[i[1] for i in posG.values()]
    df_nodepos['z']=[i[2] for i in posG.values()]
    df_nodepos.to_csv(graphname+'_nodepositions.csv', header=None, index=0)

    # NODE COLORS 
    d_node_colors = nx.get_node_attributes(G, "nodecolor")
    df_nodecol = pd.DataFrame()
    df_nodecol['r']=[hex_to_rgb(i)[0] if type(i)==str else i[0] for i in d_node_colors.values()]
    df_nodecol['g']=[hex_to_rgb(i)[1] if type(i)==str else i[1] for i in d_node_colors.values()]
    df_nodecol['b']=[hex_to_rgb(i)[2] if type(i)==str else i[2] for i in d_node_colors.values()]
    df_nodecol['a']=[100 if type(i)==str else i[3] for i in d_node_colors.values()]
    df_nodecol.to_csv(graphname+'_nodecolors.csv', header=None, index=0)

    # NODE PROPERTIES
    #annotation = nx.get_node_attributes(G, "annotation")
    #l_annotation = list(annotation.values())
    df_nodeprop = pd.DataFrame()
    df_nodeprop['prop'] = ['cluster: '+str(i)+'; node:  '+str(j) for i,j in zip(dict_for_cluster.values(), dict_for_cluster.keys())]
    df_nodeprop.to_csv(graphname+'_nodeproperties.csv', header=None, index=0)

    # LINKS
    df_links = pd.DataFrame()
    df_links['start'] = [i[0] for i in list(G.edges())]
    df_links['end'] = [i[1] for i in list(G.edges())]
    df_links.to_csv(graphname+'_links.csv', header=None, index=0)

    # LINK COLORS
    linkcolor = '#909090'
    nx.set_edge_attributes(G, linkcolor, name="linkcolor")
    linkcolor = nx.get_edge_attributes(G,name="linkcolor")
    df_linkcol = pd.DataFrame()
    df_linkcol['r']=[hex_to_rgb(i)[0] if type(i)==str else i[0] for i in linkcolor.values()]
    df_linkcol['g']=[hex_to_rgb(i)[1] if type(i)==str else i[1] for i in linkcolor.values()]
    df_linkcol['b']=[hex_to_rgb(i)[2] if type(i)==str else i[2] for i in linkcolor.values()]
    df_linkcol['a']=[80 if type(i)==str else i[3] for i in linkcolor.values()]
    df_linkcol.to_csv(graphname+'_linkcolors.csv', header=None, index=0)

    # CLUSTER LABELS
    clusters = nx.get_node_attributes(G, "cluster")
    clustercolors = sns.color_palette("Set1").as_hex()[:len(set(clusters.values()))]
    d_clustercolors = dict(zip(set(clusters.values()),clustercolors)) 
    clusterlabels = []
    for cluster in d_clustercolors.keys():
        sublist = []
        for k,v in dict_for_cluster.items():
            if cluster == v:
                sublist.append(str(k))
        sublist.insert(0,str(cluster))
        clusterlabels.append(sublist)

    with open(graphname+'_clusterlabelsTEST.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in clusterlabels:
            csvwriter.writerow(row)